# Compare neighbourhoods

In [1]:
import scanpy as sc
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
import random
import scipy as scipy
import scrabbit

## Load data

In [2]:
m_data = sc.read_h5ad("../data-in/mouse/anndata.h5ad")
m_data

AnnData object with n_obs × n_vars = 430339 × 23972
    obs: 'sample', 'stage', 'stage.mapped', 'celltype', 'celltype.extended', 'tube_name', 'somite_count', 'cluster', 'cluster.sub', 'louvain', 'leiden', 'celltype.clustering', 'day'
    var: 'ensembl_id'
    uns: 'celltype_colors', 'draw_graph', 'leiden', 'louvain', 'neighbors', 'pca', 'umap'
    obsm: 'X_draw_graph_fa', 'X_pca', 'X_umap'
    varm: 'PCs'
    layers: 'counts'
    obsp: 'connectivities', 'distances'

In [3]:
m_data.var["gene_name"] = m_data.var.index
m_data.var.index = m_data.var["ensembl_id"]
m_data.var

,ensembl_id,gene_name
ensembl_id,,
ENSMUSG00000001138,ENSMUSG00000001138,Cnnm3
ENSMUSG00000001143,ENSMUSG00000001143,Lman2l
ENSMUSG00000001674,ENSMUSG00000001674,Ddx18
ENSMUSG00000002459,ENSMUSG00000002459,Rgs20
ENSMUSG00000002881,ENSMUSG00000002881,Nab1
...,...,...
ENSMUSG00000101966,ENSMUSG00000101966,Gm5248
ENSMUSG00000105709,ENSMUSG00000105709,Gm42723
ENSMUSG00000108889,ENSMUSG00000108889,Olfr1555-ps1


In [112]:
# Load mouse counts data
m_counts = sc.read_mtx("../data-in/mouse/counts.mtx")
m_data.layers["counts"] = np.transpose(m_counts.X)

AnnData object with n_obs × n_vars = 23972 × 430339

In [4]:
# Load rabbit data
r_data = sc.read_h5ad("../data-in/rabbit/anndata.h5ad")
r_data

AnnData object with n_obs × n_vars = 146133 × 30725
    obs: 'cell', 'barcode', 'sample', 'stage', 'batch', 'doub.density', 'doublet', 'stripped', 'cluster', 'cluster.sub', 'cluster.stage', 'cluster.theiler', 'sizeFactor', 'celltype', 'singler', 'leiden_res1', 'leiden_res2', 'leiden_res1_5', 'leiden_res2_5', 'leiden_res3', 'leiden_res5', 'leiden_res10', 'leiden_res6', 'leiden_res7', 'leiden_res8', 'anatomical_loc', 'day', 'celltype_old'
    var: 'ensembl_ids'
    uns: 'anatomical_loc_colors', 'celltype_colors', 'celltype_old_colors', 'draw_graph', 'leiden', 'neighbors', 'singler_colors', 'umap'
    obsm: 'X_draw_graph_fa', 'X_pca', 'X_tsne', 'X_umap'
    layers: 'logcounts'
    obsp: 'connectivities', 'distances'

In [5]:
# Load one-to-one orthologs
rm_orthologs = pd.read_csv("../data-in/orthologs/mmusculus.tsv", sep="\t")
rm_orthologs

,ref,query
ENSOCUG00000000006,ENSOCUG00000000006,ENSMUSG00000026102
ENSOCUG00000000007,ENSOCUG00000000007,ENSMUSG00000028480
ENSOCUG00000000008,ENSOCUG00000000008,ENSMUSG00000070999
ENSOCUG00000000009,ENSOCUG00000000009,ENSMUSG00000028478
ENSOCUG00000000010,ENSOCUG00000000010,ENSMUSG00000028479
...,...,...
ENSOCUG00000039241,ENSOCUG00000039241,ENSMUSG00000022255
ENSOCUG00000039281,ENSOCUG00000039281,ENSMUSG00000024885
ENSOCUG00000039392,ENSOCUG00000039392,ENSMUSG00000014852
ENSOCUG00000039553,ENSOCUG00000039553,ENSMUSG00000009566


In [6]:
r_cellcycle = pd.read_csv("../data-in/rabbit/cc_genes.tsv", sep="\t")
m_cellcycle = pd.read_csv("../data-in/mouse/cc_genes.tsv", sep="\t")

In [7]:
r_mito = pd.read_csv("../data-in/rabbit/mito_genes.tsv", sep="\t")
m_mito = pd.read_csv("../data-in/mouse/mito_genes.tsv", sep="\t")

## Load neighbourhood data

In [8]:
r_nhoods = sc.read_mtx("../data-out/compare_neighbourhoods/r_nhoods.mtx")
r_nhoods

AnnData object with n_obs × n_vars = 146133 × 5253

In [9]:
m_nhoods = sc.read_mtx("../data-out/compare_neighbourhoods/m_nhoods.mtx")
m_nhoods

AnnData object with n_obs × n_vars = 430339 × 14034

## Grid search

In [ ]:
norm_methods = ["size_factors"]
hvg_methods = ["seurat", "edgeR"]
join_type = ["intersect", "union"]
n_genes = [500, 1000, 2000, 2500, 3000, 5000]
sim_measure = ["pearson", "spearman"]

### Compute gene specificity

TODO: Make this reproducible

In [10]:
import warnings

# nan entries correspond to non-expressed genes, can be removed prior
def calcGeneSpecificity(X,group_by=None, mask=None):
    if(mask is None):
        mask = pd.get_dummies(adata.obs[group_by])
        
    ncells=mask.sum(0)
    ncells_array = np.squeeze(np.asarray(1/ncells))
    ncells_mat = scipy.sparse.diags(ncells_array)
    mask = scipy.sparse.csr_matrix(mask).T

    ctype_sum = mask@X
    ctype_mean = ncells_mat@ctype_sum
    
    N = ctype_mean.shape[0]
    row_sums = np.squeeze(np.asarray(ctype_mean.sum(axis=1)))[:,None]
    gspec = ctype_mean/(row_sums/N) #TODO: make this sparse    
    
    return(gspec)



def filterGenes(adata, orthologs, min_counts=1, n_hvgs=2000, cc_genes=None, mito_genes=None, layer=None):
    
    # Filter one-to-one orthologs
    filt = adata[:,adata.var.index.isin(orthologs)]
    
    # Filter min counts 
    exp_genes = sc.pp.filter_genes(filt,min_counts=min_counts,inplace=False)
    filt = filt[:,exp_genes[0]]
    
    # Remove cell-cycle genes
    if(cc_genes is not None): filt = filt[:,~filt.var.index.isin(cc_genes)]
    
    # Remove mitochondrial genes
    if(mito_genes is not None): filt = filt[:,~filt.var.index.isin(mito_genes)]
    
    # Filter top highly variable genes
    if(n_hvgs):
        sc.pp.highly_variable_genes(filt,n_top_genes=n_hvgs, layer=layer)
        filt = filt[:,filt.var["highly_variable"]]
           
    return(filt)


def combineGenes(r_filt, m_filt, orthologs, join_type="intersect"):
    r_genes = r_filt.var.index
    m_genes = m_filt.var.index  

    # Assumes orthologs[,0] is rabbit, orthologs[,1] is mouse
    out_genes = []
    if(join_type=="union"): 
        out_genes = orthologs.loc[orthologs.iloc[:,0].isin(r_genes) | orthologs.iloc[:,1].isin(m_genes),:]

    elif(join_type == "intersect"):
        out_genes = orthologs.loc[orthologs.iloc[:,0].isin(r_genes) & orthologs.iloc[:,1].isin(m_genes),:]
    else:
        warnings.warn("join_type must be one of 'union' or 'intersect'.")

    return(out_genes)


In [135]:
# Filter rabbit genes
r_filt = filterGenes(r_data, orthologs=rm_orthologs["ref"], n_hvgs=2000,
                     cc_genes=r_cellcycle["ensembl_gene_id"], mito_genes=r_mito["ensembl_gene_id"], 
                     layer="logcounts")
r_filt 

/hps/software/users/marioni/dkeitley/miniconda3/envs/pyenv/lib/python3.8/site-packages/scanpy/preprocessing/_utils.py:11: RuntimeWarning: overflow encountered in multiply
  mean_sq = np.multiply(X, X).mean(axis=axis, dtype=np.float64)
/hps/software/users/marioni/dkeitley/miniconda3/envs/pyenv/lib/python3.8/site-packages/scanpy/preprocessing/_utils.py:12: RuntimeWarning: overflow encountered in square
  var = mean_sq - mean ** 2
/hps/software/users/marioni/dkeitley/miniconda3/envs/pyenv/lib/python3.8/site-packages/scanpy/preprocessing/_utils.py:12: RuntimeWarning: invalid value encountered in subtract
  var = mean_sq - mean ** 2
/hps/software/users/marioni/dkeitley/miniconda3/envs/pyenv/lib/python3.8/site-packages/scanpy/preprocessing/_highly_variable_genes.py:237: RuntimeWarning: invalid value encountered in true_divide
  df['dispersions_norm'] = (
Trying to set attribute `.uns` of view, copying.
/hps/software/users/marioni/dkeitley/miniconda3/envs/pyenv/lib/python3.8/site-packages/ann

View of AnnData object with n_obs × n_vars = 146133 × 2000
    obs: 'cell', 'barcode', 'sample', 'stage', 'batch', 'doub.density', 'doublet', 'stripped', 'cluster', 'cluster.sub', 'cluster.stage', 'cluster.theiler', 'sizeFactor', 'celltype', 'singler', 'leiden_res1', 'leiden_res2', 'leiden_res1_5', 'leiden_res2_5', 'leiden_res3', 'leiden_res5', 'leiden_res10', 'leiden_res6', 'leiden_res7', 'leiden_res8', 'anatomical_loc', 'day', 'celltype_old'
    var: 'ensembl_ids', 'mean', 'std', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'anatomical_loc_colors', 'celltype_colors', 'celltype_old_colors', 'draw_graph', 'leiden', 'neighbors', 'singler_colors', 'umap', 'hvg'
    obsm: 'X_draw_graph_fa', 'X_pca', 'X_tsne', 'X_umap'
    layers: 'logcounts'
    obsp: 'connectivities', 'distances'

In [144]:
# Filter mouse genes
m_filt = filterGenes(m_data, orthologs=rm_orthologs["query"], n_hvgs=2000,
                     cc_genes=m_cellcycle["ensembl_gene_id"], mito_genes=m_mito["ensembl_gene_id"])
m_filt 

/hps/software/users/marioni/dkeitley/miniconda3/envs/pyenv/lib/python3.8/site-packages/anndata/_core/anndata.py:1094: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if not is_categorical(df_full[k]):
Trying to set attribute `.uns` of view, copying.
/hps/software/users/marioni/dkeitley/miniconda3/envs/pyenv/lib/python3.8/site-packages/anndata/_core/anndata.py:1094: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if not is_categorical(df_full[k]):


View of AnnData object with n_obs × n_vars = 430339 × 2000
    obs: 'sample', 'stage', 'stage.mapped', 'celltype', 'celltype.extended', 'tube_name', 'somite_count', 'cluster', 'cluster.sub', 'louvain', 'leiden', 'celltype.clustering', 'day'
    var: 'ensembl_id', 'gene_name', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'celltype_colors', 'draw_graph', 'leiden', 'louvain', 'neighbors', 'pca', 'umap', 'hvg'
    obsm: 'X_draw_graph_fa', 'X_pca', 'X_umap'
    varm: 'PCs'
    layers: 'counts'
    obsp: 'connectivities', 'distances'

In [159]:
# Take intersection of rabbit/mouse filtered genes
gene_set = combineGenes(r_filt,m_filt, rm_orthologs, join_type="union")
r_final = r_data[:,gene_set["ref"]]
m_final = m_data[:,gene_set["query"]]
print("Number of genes in final gene set: " + str(gene_set.shape[0]))

Number of genes in final gene set: 3640


In [160]:
r_gspec = calcGeneSpecificity(r_final.layers["logcounts"], mask=r_nhoods.X)
r_gspec.shape

(5253, 3640)

In [161]:
m_gspec = calcGeneSpecificity(m_final.X, mask=m_nhoods.X)
m_gspec.shape

(14034, 3640)

In [162]:
gspec_cor = scipy.stats.spearmanr(r_gspec, m_gspec, axis=1)

In [163]:
N = r_gspec.shape[0]
gspec_cor = gspec_cor[0][0:N,N:]

In [55]:
gspec_cor = np.corrcoef(r_gspec,m_gspec)

In [56]:
N = r_gspec.shape[0]
gspec_cor = gspec_cor[0:N,N:]

## Export

In [164]:
# Export correlations
np.savetxt("../data-out/compare_neighbourhoods/gspec_cor.tsv", gspec_cor, delimiter="\t")

In [165]:
# Export gene specificities
r_gspec = pd.DataFrame(r_gspec, columns=r_final.var.index)
m_gspec = pd.DataFrame(m_gspec, columns=m_final.var.index)

r_gspec.to_csv("../data-out/compare_neighbourhoods/r_gspec.tsv", index=False,header=True, sep='\t')
m_gspec.to_csv("../data-out/compare_neighbourhoods/m_gspec.tsv", index=False,header=True, sep='\t')

## Rabbit-macaque test

In [61]:
# Load macaque data
mf_data = sc.read_h5ad("../data-in/datasets/Yang_2021/anndata.h5ad")
mf_data

AnnData object with n_obs × n_vars = 7194 × 19367
    obs: 'celltype', 'stage', 'batch', 'sizeFactor'
    var: 'ensembl_id', 'gene_name'
    obsm: 'X_pca', 'X_umap'
    layers: 'counts'

In [62]:
# Load rabbit-macaque orthologs
ocmf_orthologs = pd.read_csv("../data-in/orthologs/mfascicularis.tsv", sep="\t")
ocmf_orthologs

,ref,query
0,ENSOCUG00000000008,ENSMFAG00000033331
1,ENSOCUG00000000009,ENSMFAG00000034266
2,ENSOCUG00000000010,ENSMFAG00000039922
3,ENSOCUG00000000012,ENSMFAG00000031252
4,ENSOCUG00000000025,ENSMFAG00000043043
...,...,...
13583,ENSOCUG00000028824,ENSMFAG00000024901
13584,ENSOCUG00000030953,ENSMFAG00000008617
13585,ENSOCUG00000031936,ENSMFAG00000021894
13586,ENSOCUG00000031936,ENSMFAG00000022672


In [65]:
mf_nhoods = sc.read_mtx("../data-out/compare_neighbourhoods/mf_nhoods.mtx")
mf_nhoods

AnnData object with n_obs × n_vars = 7194 × 478

In [127]:
# Filter rabbit genes
r_filt = filterGenes(r_data, orthologs=ocmf_orthologs["ref"], n_hvgs=2000,
                     cc_genes=r_cellcycle["ensembl_gene_id"], mito_genes=r_mito["ensembl_gene_id"], 
                     layer="logcounts")
r_filt 

/hps/software/users/marioni/dkeitley/miniconda3/envs/pyenv/lib/python3.8/site-packages/scanpy/preprocessing/_utils.py:11: RuntimeWarning: overflow encountered in multiply
  mean_sq = np.multiply(X, X).mean(axis=axis, dtype=np.float64)
/hps/software/users/marioni/dkeitley/miniconda3/envs/pyenv/lib/python3.8/site-packages/scanpy/preprocessing/_utils.py:12: RuntimeWarning: overflow encountered in square
  var = mean_sq - mean ** 2
/hps/software/users/marioni/dkeitley/miniconda3/envs/pyenv/lib/python3.8/site-packages/scanpy/preprocessing/_utils.py:12: RuntimeWarning: invalid value encountered in subtract
  var = mean_sq - mean ** 2
/hps/software/users/marioni/dkeitley/miniconda3/envs/pyenv/lib/python3.8/site-packages/scanpy/preprocessing/_highly_variable_genes.py:237: RuntimeWarning: invalid value encountered in true_divide
  df['dispersions_norm'] = (
Trying to set attribute `.uns` of view, copying.
/hps/software/users/marioni/dkeitley/miniconda3/envs/pyenv/lib/python3.8/site-packages/ann

View of AnnData object with n_obs × n_vars = 146133 × 2000
    obs: 'cell', 'barcode', 'sample', 'stage', 'batch', 'doub.density', 'doublet', 'stripped', 'cluster', 'cluster.sub', 'cluster.stage', 'cluster.theiler', 'sizeFactor', 'celltype', 'singler', 'leiden_res1', 'leiden_res2', 'leiden_res1_5', 'leiden_res2_5', 'leiden_res3', 'leiden_res5', 'leiden_res10', 'leiden_res6', 'leiden_res7', 'leiden_res8', 'anatomical_loc', 'day', 'celltype_old'
    var: 'ensembl_ids', 'mean', 'std', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'anatomical_loc_colors', 'celltype_colors', 'celltype_old_colors', 'draw_graph', 'leiden', 'neighbors', 'singler_colors', 'umap', 'hvg'
    obsm: 'X_draw_graph_fa', 'X_pca', 'X_tsne', 'X_umap'
    layers: 'logcounts'
    obsp: 'connectivities', 'distances'

In [128]:
# Filter mouse genes
mf_filt = filterGenes(mf_data, orthologs=ocmf_orthologs["query"], n_hvgs=2000)
mf_filt 

Trying to set attribute `.uns` of view, copying.
/hps/software/users/marioni/dkeitley/miniconda3/envs/pyenv/lib/python3.8/site-packages/anndata/_core/anndata.py:1094: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if not is_categorical(df_full[k]):


View of AnnData object with n_obs × n_vars = 7194 × 2000
    obs: 'celltype', 'stage', 'batch', 'sizeFactor'
    var: 'ensembl_id', 'gene_name', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'hvg'
    obsm: 'X_pca', 'X_umap'
    layers: 'counts'

In [129]:
# Take intersection of rabbit/mouse filtered genes
gene_set = combineGenes(r_filt,mf_filt, ocmf_orthologs, join_type="intersect")
r_final = r_data[:,gene_set["ref"]]
mf_final = mf_data[:,gene_set["query"]]
print("Number of genes in final gene set: " + str(gene_set.shape[0]))

Number of genes in final gene set: 385


In [130]:
r_gspec = calcGeneSpecificity(r_final.layers["logcounts"], mask=r_nhoods.X)
r_gspec.shape

(5253, 385)

In [131]:
mf_gspec = calcGeneSpecificity(mf_final.X, mask=mf_nhoods.X)
mf_gspec.shape

(478, 385)

In [132]:
gspec_cor = np.corrcoef(r_gspec,mf_gspec)

In [133]:
N = r_gspec.shape[0]
gspec_cor = gspec_cor[0:N,N:]

In [134]:
# Export correlations
np.savetxt("../data-out/compare_neighbourhoods/macaque_test/gspec_cor.tsv", gspec_cor, delimiter="\t")

# Export gene specificities
r_gspec = pd.DataFrame(r_gspec, columns=r_final.var.index)
mf_gspec = pd.DataFrame(mf_gspec, columns=mf_final.var.index)

r_gspec.to_csv("../data-out/compare_neighbourhoods/macaque_test/r_gspec.tsv", index=False,header=True, sep='\t')
mf_gspec.to_csv("../data-out/compare_neighbourhoods/macaque_test/mf_gspec.tsv", index=False,header=True, sep='\t')

## Rabbit-human test

In [86]:
# Load human data
hs_data = sc.read_h5ad("../data-in/datasets/Tyser_2020/adata.h5ad")
hs_data

AnnData object with n_obs × n_vars = 1195 × 57490
    obs: 'celltype', 'stage'
    obsm: 'UMAP'

In [87]:
# Load rabbit-macaque orthologs
ochs_orthologs = pd.read_csv("../data-in/orthologs/hsapiens.tsv", sep="\t")
ochs_orthologs

,ref,query
0,ENSOCUG00000000006,INPP1
1,ENSOCUG00000000007,GLIPR2
2,ENSOCUG00000000008,CCIN
3,ENSOCUG00000000009,CLTA
4,ENSOCUG00000000010,GNE
...,...,...
17542,ENSOCUG00000031936,RN7SL683P
17543,ENSOCUG00000034014,MIR3618
17544,ENSOCUG00000034135,MIR507
17545,ENSOCUG00000034135,MIR506


In [88]:
hs_nhoods = sc.read_mtx("../data-out/compare_neighbourhoods/hs_nhoods.mtx")
hs_nhoods

AnnData object with n_obs × n_vars = 1195 × 139

In [113]:
# Filter rabbit genes
r_filt = filterGenes(r_data, orthologs=ochs_orthologs["ref"], n_hvgs=2000,
                     cc_genes=r_cellcycle["ensembl_gene_id"], mito_genes=r_mito["ensembl_gene_id"], 
                     layer="logcounts")
r_filt 

/hps/software/users/marioni/dkeitley/miniconda3/envs/pyenv/lib/python3.8/site-packages/anndata/_core/anndata.py:1094: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if not is_categorical(df_full[k]):
/hps/software/users/marioni/dkeitley/miniconda3/envs/pyenv/lib/python3.8/site-packages/scanpy/preprocessing/_utils.py:11: RuntimeWarning: overflow encountered in multiply
  mean_sq = np.multiply(X, X).mean(axis=axis, dtype=np.float64)
/hps/software/users/marioni/dkeitley/miniconda3/envs/pyenv/lib/python3.8/site-packages/scanpy/preprocessing/_utils.py:12: RuntimeWarning: overflow encountered in square
  var = mean_sq - mean ** 2
/hps/software/users/marioni/dkeitley/miniconda3/envs/pyenv/lib/python3.8/site-packages/scanpy/preprocessing/_utils.py:12: RuntimeWarning: invalid value encountered in subtract
  var = mean_sq - mean ** 2
/hps/software/users/marioni/dkeitley/miniconda3/envs/pyenv/lib/python3.8/site-packages/sca

View of AnnData object with n_obs × n_vars = 146133 × 2000
    obs: 'cell', 'barcode', 'sample', 'stage', 'batch', 'doub.density', 'doublet', 'stripped', 'cluster', 'cluster.sub', 'cluster.stage', 'cluster.theiler', 'sizeFactor', 'celltype', 'singler', 'leiden_res1', 'leiden_res2', 'leiden_res1_5', 'leiden_res2_5', 'leiden_res3', 'leiden_res5', 'leiden_res10', 'leiden_res6', 'leiden_res7', 'leiden_res8', 'anatomical_loc', 'day', 'celltype_old'
    var: 'ensembl_ids', 'mean', 'std', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'anatomical_loc_colors', 'celltype_colors', 'celltype_old_colors', 'draw_graph', 'leiden', 'neighbors', 'singler_colors', 'umap', 'hvg'
    obsm: 'X_draw_graph_fa', 'X_pca', 'X_tsne', 'X_umap'
    layers: 'logcounts'
    obsp: 'connectivities', 'distances'

In [114]:
# Filter mouse genes
hs_filt = filterGenes(hs_data, orthologs=ochs_orthologs["query"], n_hvgs=2000)
hs_filt 

Trying to set attribute `.uns` of view, copying.
/hps/software/users/marioni/dkeitley/miniconda3/envs/pyenv/lib/python3.8/site-packages/anndata/_core/anndata.py:1094: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if not is_categorical(df_full[k]):


View of AnnData object with n_obs × n_vars = 1195 × 2000
    obs: 'celltype', 'stage'
    var: 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'hvg'
    obsm: 'UMAP'

In [115]:
# Take intersection of rabbit/mouse filtered genes
gene_set = combineGenes(r_filt,hs_filt, ochs_orthologs, join_type="intersect")
r_final = r_data[:,gene_set["ref"]]
hs_final = hs_data[:,gene_set["query"]]
print("Number of genes in final gene set: " + str(gene_set.shape[0]))

Number of genes in final gene set: 369


In [116]:
r_gspec = calcGeneSpecificity(r_final.layers["logcounts"], mask=r_nhoods.X)
r_gspec.shape

(5253, 369)

In [117]:
hs_gspec = calcGeneSpecificity(hs_final.X, mask=hs_nhoods.X)
hs_gspec.shape

(139, 369)

In [118]:
gspec_cor = np.corrcoef(r_gspec,hs_gspec)

In [119]:
N = r_gspec.shape[0]
gspec_cor = gspec_cor[0:N,N:]

In [121]:
# Export correlations
np.savetxt("../data-out/compare_neighbourhoods/human_test/gspec_cor.tsv", gspec_cor, delimiter="\t")

# Export gene specificities
r_gspec = pd.DataFrame(r_gspec, columns=r_final.var.index)
hs_gspec = pd.DataFrame(hs_gspec, columns=hs_final.var.index)

r_gspec.to_csv("../data-out/compare_neighbourhoods/human_test/r_gspec.tsv", index=False,header=True, sep='\t')
hs_gspec.to_csv("../data-out/compare_neighbourhoods/human_test/hs_gspec.tsv", index=False,header=True, sep='\t')

In [123]:
r_filt.X[1:5,1:5].todense()

matrix([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]], dtype=float32)

0.5550517437408959

## Find genes with large differences in gene specificity

In [44]:
r_celltypes = pd.read_csv("../data-in/rabbit/annotations_12-10-21.tsv",sep="\t",index_col=0)
r_data.obs["celltype"] = r_celltypes["assigned_celltype"]

In [35]:
r_gspec.shape

(5253, 2200)

In [36]:
m_gspec.shape

(14034, 2200)

In [28]:
m_val.shape

(2200, 14034)

In [31]:
np.mean(r_gspec,axis=1).shape

(5253, 1)

In [ ]:
r_val = (r_gspec-np.mean(r_gspec,axis=1))
m_val = (m_gspec.transpose()-np.mean(m_gspec,axis=1)).transpose()
vals = np.absolute(r_val-m_val)

In [ ]:
top = np.argpartition(vals, -5, axis=1)[:,(-5):]
#bottom = np.argpartition(vals, 5, axis=1)[:, :5]

In [ ]:
m_filt.var
m_filt.var.iloc[top[list(ctype_list).index("Epiblast"),:]]

## Test R gene spec code

In [30]:
r_data.var

,ensembl_ids
ENSOCUG00000014251,ENSOCUG00000014251
ENSOCUG00000005054,ENSOCUG00000005054
ENSOCUG00000005046,ENSOCUG00000005046
ENSOCUG00000005044,ENSOCUG00000005044
ENSOCUG00000005040,ENSOCUG00000005040
...,...
ENSG00000172508,ENSG00000172508
ENSG00000283268,ENSG00000283268
ENSG00000067066,ENSG00000067066
ENSG00000175602,ENSG00000175602


In [38]:
rm_orthologs["ref"][1:20]

ENSOCUG00000000007    ENSOCUG00000000007
ENSOCUG00000000008    ENSOCUG00000000008
ENSOCUG00000000009    ENSOCUG00000000009
ENSOCUG00000000010    ENSOCUG00000000010
ENSOCUG00000000025    ENSOCUG00000000025
ENSOCUG00000000031    ENSOCUG00000000031
ENSOCUG00000000032    ENSOCUG00000000032
ENSOCUG00000000034    ENSOCUG00000000034
ENSOCUG00000000044    ENSOCUG00000000044
ENSOCUG00000000046    ENSOCUG00000000046
ENSOCUG00000000049    ENSOCUG00000000049
ENSOCUG00000000051    ENSOCUG00000000051
ENSOCUG00000000060    ENSOCUG00000000060
ENSOCUG00000000068    ENSOCUG00000000068
ENSOCUG00000000079    ENSOCUG00000000079
ENSOCUG00000000111    ENSOCUG00000000111
ENSOCUG00000000114    ENSOCUG00000000114
ENSOCUG00000000117    ENSOCUG00000000117
ENSOCUG00000000119    ENSOCUG00000000119
Name: ref, dtype: object

In [46]:
r_final = r_data[:,rm_orthologs["ref"][0:20]]
r_final

View of AnnData object with n_obs × n_vars = 146133 × 20
    obs: 'cell', 'barcode', 'sample', 'stage', 'batch', 'doub.density', 'doublet', 'stripped', 'cluster', 'cluster.sub', 'cluster.stage', 'cluster.theiler', 'sizeFactor', 'celltype', 'singler', 'leiden_res1', 'leiden_res2', 'leiden_res1_5', 'leiden_res2_5', 'leiden_res3', 'leiden_res5', 'leiden_res10', 'leiden_res6', 'leiden_res7', 'leiden_res8', 'anatomical_loc', 'day', 'celltype_old'
    var: 'ensembl_ids'
    uns: 'anatomical_loc_colors', 'celltype_colors', 'celltype_old_colors', 'draw_graph', 'leiden', 'neighbors', 'singler_colors', 'umap'
    obsm: 'X_draw_graph_fa', 'X_pca', 'X_tsne', 'X_umap'
    layers: 'logcounts'
    obsp: 'connectivities', 'distances'

In [45]:
m_final = m_data[:, rm_orthologs["query"][0:20]]
m_final

View of AnnData object with n_obs × n_vars = 430339 × 20
    obs: 'sample', 'stage', 'stage.mapped', 'celltype', 'celltype.extended', 'tube_name', 'somite_count', 'cluster', 'cluster.sub', 'louvain', 'leiden', 'celltype.clustering', 'day'
    var: 'ensembl_id', 'gene_name'
    uns: 'celltype_colors', 'draw_graph', 'leiden', 'louvain', 'neighbors', 'pca', 'umap'
    obsm: 'X_draw_graph_fa', 'X_pca', 'X_umap'
    varm: 'PCs'
    layers: 'counts'
    obsp: 'connectivities', 'distances'

In [ ]:
# nan entries correspond to non-expressed genes, can be removed prior
def calcGeneSpecificity(X,group_by=None, mask=None):
    if(mask is None):
        mask = pd.get_dummies(adata.obs[group_by])
        
    ncells=mask.sum(0)
    ncells_array = np.squeeze(np.asarray(1/ncells))
    ncells_mat = scipy.sparse.diags(ncells_array)
    mask = scipy.sparse.csr_matrix(mask).T

    ctype_sum = mask@X
    ctype_mean = ncells_mat@ctype_sum
    
    N = ctype_mean.shape[0]
    row_sums = np.squeeze(np.asarray(ctype_mean.sum(axis=1)))[:,None]
    gspec = ctype_mean/(row_sums/N) #TODO: make this sparse    
    
    return(gspec)


In [52]:
mask = r_nhoods.X

In [53]:
ncells=mask.sum(0)
ncells_array = np.squeeze(np.asarray(1/ncells))
ncells_mat = scipy.sparse.diags(ncells_array)
mask = scipy.sparse.csr_matrix(mask).T

In [49]:
ctype_sum = mask@r_final.layers["logcounts"]

In [54]:
ctype_sum

<5253x20 sparse matrix of type '<class 'numpy.float64'>'
	with 50906 stored elements in Compressed Sparse Column format>

In [55]:
ctype_mean = ncells_mat@ctype_sum

In [56]:
ctype_mean

<5253x20 sparse matrix of type '<class 'numpy.float64'>'
	with 50906 stored elements in Compressed Sparse Row format>

In [57]:
ctype_mean.todense()[0:5,0:5]

matrix([[0.02174117, 0.0262958 , 0.        , 2.8303215 , 0.0691548 ],
        [0.10012705, 0.0059232 , 0.        , 2.84550883, 0.03163389],
        [0.0120541 , 0.00803434, 0.        , 2.84465638, 0.04731635],
        [0.17091167, 0.        , 0.        , 4.1010592 , 0.02162497],
        [0.03250837, 0.00506926, 0.        , 2.6556218 , 0.09810943]])

In [58]:
N = ctype_mean.shape[0]
row_sums = np.squeeze(np.asarray(ctype_mean.sum(axis=1)))[:,None]
gspec = ctype_mean/(row_sums/N)

In [63]:
gspec

matrix([[  34.56182386,   41.8022879 ,    0.        , ...,   89.48142613,
           22.40561622,    0.        ],
        [ 164.55089172,    9.73430673,    0.        , ...,   12.1203683 ,
            0.        ,    0.        ],
        [  19.42005894,   12.94392909,    0.        , ...,   30.07524395,
            0.        ,    0.        ],
        ...,
        [ 319.93867928,    0.        ,    0.        , ...,    0.        ,
           24.39619234,    0.        ],
        [1017.77826619,    0.        ,    0.        , ...,    0.        ,
            0.        ,    0.        ],
        [1141.74208663,    0.        ,    0.        , ...,    0.        ,
            0.        ,    0.        ]])

In [64]:
gspec.shape

(5253, 20)

In [65]:
r_gspec = calcGeneSpecificity(r_final.layers["logcounts"], mask=r_nhoods.X)
r_gspec.shape

(5253, 20)

In [67]:
m_gspec = calcGeneSpecificity(m_final.X, mask=m_nhoods.X)
m_gspec.shape

(14034, 20)

In [68]:
gspec_cor = np.corrcoef(r_gspec,m_gspec)

In [73]:
N = r_gspec.shape[0]
gspec_cor = gspec_cor[0:N,N:]

In [74]:
gspec_cor.shape

(5253, 14034)

In [75]:
gspec_cor[0:5,0:5]

array([[0.94510542, 0.97638478, 0.95040586, 0.92983958, 0.95500062],
       [0.93911609, 0.97348843, 0.94451899, 0.92378198, 0.94850207],
       [0.93803041, 0.97465299, 0.94522356, 0.9230287 , 0.94944886],
       [0.91957246, 0.96008133, 0.92606233, 0.90435461, 0.93066863],
       [0.94252549, 0.97895595, 0.9496059 , 0.9263779 , 0.9512499 ]])